In [53]:
import numpy as np
from osgeo import gdal
#import sys
#import matplotlib.mlab as mlab
#import matplotlib.pyplot as plt
import scipy
from scipy import signal
from scipy.misc import toimage
import cv2
from PIL import Image, ImageEnhance
import statistics

In [ ]:
ds=gdal.Open('FaisullaganjDay1topright_Elevation_export_SunApr22.tif')
print("[RASTER BAND COUNT]:",ds.RasterCount)
s1 = ds.GetRasterBand(1).ReadAsArray()
print(s1.shape)

In [ ]:
print(np.amax(s1))
print(np.amin(s1))

In [ ]:
from scipy import signal
s3=scipy.signal.medfilt(s1[2500:5000,2500:5000], kernel_size=None)

In [ ]:
from scipy.misc import toimage
#toimage(s3).show()
scipy.misc.imsave('median_filter.jpg',toimage(s3))

In [ ]:
import cv2
import numpy as np

In [ ]:
alpha = float(input('* Enter the alpha value [1.0-3.0]: ')) # Simple contrast control
beta = int(input('Enter the beta value [0-100]: ')) # Simple brightness control


In [ ]:
#from scipy.misc import toimage
print (" Basic Linear Transforms ")
print ("-----------------------------")
#img=toimage( np.array(s1))
img = cv2.imread('crop_img.png')
#print(np.amax(img))
#print(np.amin(img))
#print(im.shape)
gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imwrite( "Gray_Image.jpg",gray_image)


In [ ]:
print(img.shape)
print(np.amax(img))
print(np.amin(img))

In [ ]:
#mul_img = cv2.multiply(gray_image, np.array([alpha])) # mul_img = img*alpha
mul_img = cv2.multiply(img, np.array([alpha])) # mul_img = img*alpha
new_img = cv2.add(mul_img, beta) # new_img = img*alpha + beta
#cv2.imwrite( "contrast.jpg",new_img)

In [2]:
#th, bw = cv2.threshold(new_img, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
#edges = cv2.Canny(new_img, th/2, th)
edges = cv2.Canny(new_img,500,500)
#cv2.imwrite( "Cannyedges.jpg",edges)
#edge_filter=scipy.signal.medfilt(edges, kernel_size=None)
kernel = np.ones((3,3),np.uint8)
kernel1 = np.ones((3,3),np.uint8)
#edges1 = cv2.erode(edges,kernel,iterations = 1)#erosion
edges2 = cv2.dilate(edges,kernel,iterations = 3)#dilation
edges3 = cv2.morphologyEx(edges2, cv2.MORPH_OPEN, kernel1)#opening
#edges_f = cv2.morphologyEx(edges3, cv2.MORPH_CLOSE, kernel1)#closing



cv2.imwrite( "edges.jpg",edges3)
edges11 = cv2.erode(edges3,kernel,iterations = 3)#erosion
#edges12 = cv2.erode(edges11,kernel,iterations = 1)#erosion
#edges1 = cv2.erode(edges12,kernel,iterations = 1)#erosion
#edges2 = cv2.erode(edges1,kernel,iterations = 1)
cv2.imwrite( "edges23.jpg",edges11)

NameError: name 'new_img' is not defined

In [ ]:
cv2.imwrite( "Gray_Image2.jpg",new_img)
cv2.imwrite( "edge.jpg",edges)
cv2.imwrite( "edge_filter.jpg",gray_image)

In [2]:
image = cv2.imread('edges23.jpg')
#print(image)

blurred = cv2.pyrMeanShiftFiltering(image, 31, 91)

gray = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)

ret, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

_, contours, _ = cv2.findContours(threshold, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

print("number of contours detected %d ->", len(contours))

cv2.drawContours(image, contours, -1, (0,0,255), 6)

#cv2.namedWindow('Display', cv2.WINDOW_NORMAL)

cv2.imwrite('countour1.jpg', image)

#cv2.waitKey()
#cv2.destroyWindow('Display')

number of contours detected %d -> 1172


True

In [4]:

myarray = np.asarray(contours)
print(len(myarray[1]))
print(myarray[1])

10
[[[ 738 2498]]

 [[ 738 2499]]

 [[ 739 2499]]

 [[ 740 2499]]

 [[ 741 2499]]

 [[ 742 2499]]

 [[ 742 2498]]

 [[ 741 2498]]

 [[ 740 2498]]

 [[ 739 2498]]]


In [5]:
print(contours[1].shape)
print(contours[0][0][0])

(10, 1, 2)
[2102 2498]


In [3]:
centeroid = np.zeros(shape=(len(contours),2))

In [14]:
import random
from shapely.geometry import Point, Polygon
def get_random_point_in_polygon(poly):
    (minx, miny, maxx, maxy) = poly.bounds
    while True:
        p = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if poly.contains(p):
            return p

In [34]:
import math
#image1 = np.ones((1000,1000,3), np.uint8)
#print(image1.shape)
#myarray = np.asarray(contours)
i=0
j=0
allcoords=[];
for i in range(len(contours)):
    mycoords=[];
    k,l,m=contours[i].shape
    x=0
    y=0
    for j in range(k):
        #image1[math.ceil(contours[i][j][0][0]), math.ceil(contours[i][j][0][1]),0]=255;
        #image1[math.ceil(contours[i][j][0][0]), math.ceil(contours[i][j][0][1]),1]=0;
        #image1[math.ceil(contours[i][j][0][0]), math.ceil(contours[i][j][0][1]),2]=0;
        mycoords.extend([(contours[i][j][0][0], contours[i][j][0][1])])
        x=x+contours[i][j][0][0];
        y=y+contours[i][j][0][1];
    centeroid[i,0]=math.ceil(x/k);
    
    
    
    if len(mycoords)>20:
        p1=Polygon(mycoords)
        (minx, miny, maxx, maxy) = p1.bounds
        if (p1.area/((maxx-minx)*(maxy-miny)))>0.05:
            allcoords.extend([mycoords])
        
    
    
    
    #poly = Polygon(mycoords)
    #point_in_poly = get_random_point_in_polygon(poly)
    #print(point_in_poly)
    #print(poly.centroid)
    centeroid[i,1]=math.ceil(y/k);
print(len(allcoords))


645


In [31]:
i=199
print((allcoords[i]))
p=Polygon(allcoords[i])
print(p.area)
(minx, miny, maxx, maxy) = p.bounds
print(minx, miny, maxx, maxy)
max_area=(maxx-minx)*(maxy-miny)
print(max_area)
print(p.area/max_area)
pt = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
print(pt)
pt.within(p)
#point_in_poly = get_random_point_in_polygon(p)
#print(point_in_poly)

288.0
1545.0 701.0 1624.0 745.0
3476.0
0.08285385500575373
POINT (1616.303937580631 735.6658700008842)


True

In [36]:
#calculate 5 interior points of every polyguns
all_intr_coords=[];
no_of_intr_pts=100;
for i in range(0,len(allcoords)):
    print("contour no.:",i)
    intr_coords=[];
    p=Polygon(allcoords[i])
    for j in range(0,no_of_intr_pts):
        #print("point no.:",j)
        
        point_in_poly = get_random_point_in_polygon(p)
        intr_coords.extend([(math.ceil(point_in_poly.x), math.ceil(point_in_poly.y))])
    all_intr_coords.extend([intr_coords])
print(len(all_intr_coords))

contour no.: 0
contour no.: 1
contour no.: 2
contour no.: 3
contour no.: 4
contour no.: 5
contour no.: 6
contour no.: 7
contour no.: 8
contour no.: 9
contour no.: 10
contour no.: 11
contour no.: 12
contour no.: 13
contour no.: 14
contour no.: 15
contour no.: 16
contour no.: 17
contour no.: 18
contour no.: 19
contour no.: 20
contour no.: 21
contour no.: 22
contour no.: 23
contour no.: 24
contour no.: 25
contour no.: 26
contour no.: 27
contour no.: 28
contour no.: 29
contour no.: 30
contour no.: 31
contour no.: 32
contour no.: 33
contour no.: 34
contour no.: 35
contour no.: 36
contour no.: 37
contour no.: 38
contour no.: 39
contour no.: 40
contour no.: 41
contour no.: 42
contour no.: 43
contour no.: 44
contour no.: 45
contour no.: 46
contour no.: 47
contour no.: 48
contour no.: 49
contour no.: 50
contour no.: 51
contour no.: 52
contour no.: 53
contour no.: 54
contour no.: 55
contour no.: 56
contour no.: 57
contour no.: 58
contour no.: 59
contour no.: 60
contour no.: 61
contour no.: 62
co

contour no.: 491
contour no.: 492
contour no.: 493
contour no.: 494
contour no.: 495
contour no.: 496
contour no.: 497
contour no.: 498
contour no.: 499
contour no.: 500
contour no.: 501
contour no.: 502
contour no.: 503
contour no.: 504
contour no.: 505
contour no.: 506
contour no.: 507
contour no.: 508
contour no.: 509
contour no.: 510
contour no.: 511
contour no.: 512
contour no.: 513
contour no.: 514
contour no.: 515
contour no.: 516
contour no.: 517
contour no.: 518
contour no.: 519
contour no.: 520
contour no.: 521
contour no.: 522
contour no.: 523
contour no.: 524
contour no.: 525
contour no.: 526
contour no.: 527
contour no.: 528
contour no.: 529
contour no.: 530
contour no.: 531
contour no.: 532
contour no.: 533
contour no.: 534
contour no.: 535
contour no.: 536
contour no.: 537
contour no.: 538
contour no.: 539
contour no.: 540
contour no.: 541
contour no.: 542
contour no.: 543
contour no.: 544
contour no.: 545
contour no.: 546
contour no.: 547
contour no.: 548
contour no.: 5

In [52]:
print(all_intr_coords[2][5][0])
print(all_intr_coords[2][5][0])
print(all_intr_coords[2][5])

359
359
(359, 2496)


In [ ]:
print(centeroid[1:5])

In [42]:

ds=gdal.Open('FaisullaganjDay1topright_Elevation_export_FriApr27.tif')
print("[RASTER BAND COUNT]:",ds.RasterCount)
s1 = ds.GetRasterBand(1).ReadAsArray()

[RASTER BAND COUNT]: 1


In [44]:
hgt=s1[2500:5000,2500:5000]
print(hgt.shape)
print(np.amax(hgt))
print(np.amin(hgt))
print(hgt[56,654])

(2500, 2500)
21.037134
-32767.0
5.0222087


In [41]:
print(hgt[236,555])

255


In [64]:
#height of contour
hgt_cont=[];
for i in range(len(allcoords)):
    hgt_temp=[];
    
    for j in range(no_of_intr_pts):
        hgt_temp.extend([hgt[all_intr_coords[i][j][0],all_intr_coords[i][j][1]]])
    h=statistics.median(hgt_temp)
    if h<0:
        print(i)
    hgt_cont.extend([h])    
print((hgt_cont[9]))        
        
        
    

22
26
43
108
122
125
133
151
162
199
3.981630325317383


In [71]:
print((all_intr_coords[199][1][0]))

7


In [78]:
j=122
for i in range(len(all_intr_coords[j])):
    print(hgt[all_intr_coords[j][i][0],all_intr_coords[j][i][1]])
    

-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
-32767.0
